In [1]:
# From: https://www.kaggle.com/c/dog-breed-identification/data
# Author: Morpheus Hsieh

from __future__ import print_function, division

import os, sys
import copy
import io
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
from mpl_toolkits.axes_grid1 import ImageGrid
from os import listdir
from os.path import join, isfile
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import datasets, models, transforms, utils

print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.5.1
Torchvision Version:  0.6.1


In [7]:
# Load parameters from json file

OutPath = r'D:\GitWork\dog_breed\output'
fname = 'Params_20200923-1604.json'
json_file = join(OutPath, fname_params)

with open(json_file) as fin: 
    Params = json.load(fin) 

print(json.dumps(Params, indent=4))

{
    "DataPath": "D:\\GitWork\\dog_breed\\data",
    "OutPath": "D:\\GitWork\\dog_breed\\output",
    "ProcPath": "D:\\GitWork\\dog_breed\\processed",
    "PreTrainPath": "D:\\GitWork\\dog_breed\\pretrained",
    "PreTrainFile": "resnet50_20200923-1554_acc77.pth",
    "LoadPreModel": true,
    "TestPath": "D:\\GitWork\\dog_breed\\data\\test",
    "TrainPath": "D:\\GitWork\\dog_breed\\data\\train",
    "CsvLabel": "labels.csv",
    "BatchSize": 16,
    "FracForTrain": 0.8,
    "NumClasses": 120
}

Num classes: 120


In [8]:
# Read labels information

DataPath = Params.get('DataPath')
csv_labels = Params.get('CsvLabel')
f_abspath = join(DataPath, csv_labels)

df_labels = pd.read_csv(f_abspath)

print(df_labels.info())
print()
print(df_labels.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10222 entries, 0 to 10221
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      10222 non-null  object
 1   breed   10222 non-null  object
dtypes: object(2)
memory usage: 159.8+ KB
None

                                 id             breed
0  000bec180eb18c7604dcecc8fe0dba07       boston_bull
1  001513dfcb2ffafc82cccf4d8bbaba97             dingo
2  001cdf01b096e06d78e9e5112d419397          pekinese
3  00214f311d5d2247d5dfe4fe24b2303d          bluetick
4  0021f9ceb3235effd7fcde7f7538ed62  golden_retriever
